In [ ]:
!pip install opencv-python

In [ ]:
import os
import numpy as np
import pickle
import torch
import torchvision as tv
import torch.nn.functional as F
import torch.nn as nn

from tqdm import tqdm
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
# Config model
T = 300  # Max diffusion step
max_seq_length = 5
vocab_size = 27  # ' ': 0, 'a':1, ..., 'z':26
hidden_dim = 256

# device = "xla:0"
device = "cuda:0"
batch_size = 32

In [ ]:
# Download dataset
mnist_data = tv.datasets.MNIST(
    "./",
    download=True,
    transform=tv.transforms.Compose(
        [
            tv.transforms.ToTensor(),  # first, convert image to PyTorch tensor
        ]
    ),
)

train_data_loader = torch.utils.data.DataLoader(
    mnist_data, batch_size=batch_size, shuffle=True, num_workers=2
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.43MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.7MB/s]


In [ ]:
# Utilities
n2t_mapping = {
    0: "zero",
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
}

string_to_token = lambda x: [ord(i) - 96 for i in x]  # ' ': 0, 'a':1, ..., 'z':26
# token_to_string = lambda x: [chr(i + 96) for i in x] # We do not need it here


def convert_text_label(number_labels):
    text_labels = torch.zeros((len(number_labels), max_seq_length, vocab_size))
    for bidx, nlb in enumerate(number_labels):
        tokens = string_to_token(n2t_mapping[nlb])
        for tidx, token in enumerate(tokens):
            text_labels[bidx, tidx, token] = 1.0
    return text_labels


def show_img(img):
    cv2_imshow(img * 255)

In [ ]:
# Models for noise estimation
class ResidualConvBlock(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int, is_res: bool = False
    ) -> None:
        super().__init__()
        """
        standard ResNet style convolutional block
        """
        self.same_channels = in_channels == out_channels
        self.is_res = is_res
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.GELU(),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.is_res:
            x1 = self.conv1(x)
            x2 = self.conv2(x1)
            # this adds on correct residual in case channels have increased
            if self.same_channels:
                out = x + x2
            else:
                out = x1 + x2
            return out / 1.414
        else:
            x1 = self.conv1(x)
            x2 = self.conv2(x1)
            return x2


class UnetDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetDown, self).__init__()
        """
        process and downscale the image feature maps
        """
        layers = [ResidualConvBlock(in_channels, out_channels), nn.MaxPool2d(2)]
        self.model = nn.Sequential(
            *[ResidualConvBlock(in_channels, out_channels), nn.MaxPool2d(2)]
        )

    def forward(self, x):
        return self.model(x)


class UnetUp(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetUp, self).__init__()
        """
        process and upscale the image feature maps
        """
        layers = [
            nn.ConvTranspose2d(in_channels, out_channels, 2, 2),
            ResidualConvBlock(out_channels, out_channels),
            ResidualConvBlock(out_channels, out_channels),
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x, skip):
        x = torch.cat((x, skip), 1)
        x = self.model(x)
        return x


class EmbedFC(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(EmbedFC, self).__init__()
        """
        generic one layer FC NN for embedding things
        """
        self.input_dim = input_dim
        layers = [
            nn.Linear(input_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(-1, self.input_dim)
        return self.model(x)


class ContextUnet(nn.Module):
    def __init__(self, in_channels, n_feat=256, n_classes=10):
        super(ContextUnet, self).__init__()

        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_classes = n_classes

        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)

        self.down1 = UnetDown(n_feat, n_feat)
        self.down2 = UnetDown(n_feat, 2 * n_feat)

        self.to_vec = nn.Sequential(nn.AvgPool2d(7), nn.GELU())

        self.timeembed1 = EmbedFC(1, 2 * n_feat)
        self.timeembed2 = EmbedFC(1, 1 * n_feat)
        self.contextembed1 = nn.GRU(n_classes, 2 * n_feat, batch_first=True)
        self.contextembed2 = nn.GRU(n_classes, 1 * n_feat, batch_first=True)

        self.up0 = nn.Sequential(
            # nn.ConvTranspose2d(6 * n_feat, 2 * n_feat, 7, 7), # when concat temb and cemb end up w 6*n_feat
            nn.ConvTranspose2d(
                2 * n_feat, 2 * n_feat, 7, 7
            ),  # otherwise just have 2*n_feat
            nn.GroupNorm(8, 2 * n_feat),
            nn.ReLU(),
        )

        self.up1 = UnetUp(4 * n_feat, n_feat)
        self.up2 = UnetUp(2 * n_feat, n_feat)
        self.out = nn.Sequential(
            nn.Conv2d(2 * n_feat, n_feat, 3, 1, 1),
            nn.GroupNorm(8, n_feat),
            nn.ReLU(),
            nn.Conv2d(n_feat, self.in_channels, 3, 1, 1),
        )

    def forward(self, x, c, t, context_mask):
        # x is (noisy) image, c is context label, t is timestep,
        # context_mask says which samples to block the context on

        x = self.init_conv(x)
        down1 = self.down1(x)
        down2 = self.down2(down1)
        hiddenvec = self.to_vec(down2)

        # mask out context if context_mask == 1
        context_mask = -1 * (1 - context_mask)  # need to flip 0 <-> 1
        c = c * context_mask

        # embed context, time step
        cemb1 = self.contextembed1(c)[0].view(-1, max_seq_length, self.n_feat * 2, 1, 1)
        temb1 = self.timeembed1(t).view(-1, self.n_feat * 2, 1, 1)
        cemb2 = self.contextembed2(c)[0].view(-1, max_seq_length, self.n_feat, 1, 1)
        temb2 = self.timeembed2(t).view(-1, self.n_feat, 1, 1)

        # could concatenate the context embedding here instead of adaGN
        # hiddenvec = torch.cat((hiddenvec, temb1, cemb1), 1)

        up1 = self.up0(hiddenvec)
        # up2 = self.up1(up1, down2) # if want to avoid add and multiply embeddings
        up2 = self.up1(
            cemb1.mean(1) * up1 + temb1, down2
        )  # add and multiply embeddings
        esp_theta_w_context = self.up2(cemb2.mean(1) * up2 + temb2, down1)

        esp_theta_wo_context = x + t[:, None, None, None]

        out = self.out(torch.cat((esp_theta_w_context, esp_theta_wo_context), 1))
        return out

In [ ]:
# Noise learning - Epsilon_theta
esp_theta_model = ContextUnet(
    in_channels=1, n_feat=hidden_dim, n_classes=vocab_size
).to(device)

# Optimizer and Loss
model_params = esp_theta_model.parameters()
optimizer = torch.optim.Adam(model_params, lr=1e-4)
loss_fn = torch.nn.MSELoss()

In [ ]:
# Load saved models
# !cp "/content/drive/MyDrive/Colab Notebooks/Diffusion/contextual_model.pt" contextual_model.pt
# !cp "/content/drive/MyDrive/Colab Notebooks/Diffusion/beta_t.pkl" beta_t.pkl

# esp_theta_model.load_state_dict(torch.load("contextual_model.pt"))
# beta_t = pickle.load(open("beta_t.pkl", "rb")).to(device)

In [ ]:
# Setting hyperparameters for diffusion process
beta_1 = 1e-4
beta_2 = 2e-2
beta_t = (beta_2 - beta_1) * torch.arange(0, T + 1, dtype=torch.float32) / T + beta_1
beta_t = beta_t.to(device)

alpha_t = 1 - beta_t
alphabar_t = torch.exp(torch.cumsum(torch.log(alpha_t), dim=0))

sqrtab = torch.sqrt(alphabar_t)
sqrtmab = torch.sqrt(1 - alphabar_t)

sqrt_beta_t = torch.sqrt(beta_t)
oneover_sqrta = 1 / torch.sqrt(alpha_t)
mab_over_sqrtmab = (1 - alpha_t) / sqrtmab

In [ ]:
# TRAINING
EPOCHS = 1

for e in range(EPOCHS):
    print("========== EPOCH %d ==========" % e)
    total_loss = 0
    batch_idx = 0

    for imgs, lbs in tqdm(train_data_loader, desc=f"Epoch {e + 1}"):
        # Input preparation
        x_in = imgs.to(device)  # batch, img_size, img_size
        context = convert_text_label(lbs.numpy().tolist()).to(
            device
        )  # batch, max_length, onehot_size

        # Forward diffusion
        ## Posterior prob calculation
        t = torch.randint(low=1, high=T, size=(x_in.shape[0],)).to(device)  # batch
        esp = torch.randn_like(x_in).to(device)  # batch, img_size, img_size

        x_t = sqrtab[t, None, None, None] * x_in + sqrtmab[t, None, None, None] * esp

        ## Prior prob calculation
        context_mask = torch.bernoulli(torch.zeros_like(context) + 0.1).to(device)
        esp_theta = esp_theta_model(
            x_t, context, t / T, context_mask
        )  # batch, img_size, img_size

        # Loss calculation
        loss = loss_fn(esp, esp_theta)
        total_loss += loss.cpu().item()
        if batch_idx % 100 == 0:
            print("\tLoss:", loss.item())

        # Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Increase batch idx
        batch_idx += 1

========== EPOCH 0 ==========


Epoch 1:   0%|          | 0/1875 [00:00<?, ?it/s]

	Loss: 1.1122623682022095


Epoch 1:   5%|▌         | 102/1875 [00:17<04:37,  6.39it/s]

	Loss: 0.07336524873971939


...

Epoch 1:  91%|█████████ | 1702/1875 [04:41<00:28,  6.02it/s]

	Loss: 0.03867202252149582


Epoch 1:  96%|█████████▌| 1802/1875 [04:58<00:12,  6.00it/s]

	Loss: 0.025914382189512253


Epoch 1: 100%|██████████| 1875/1875 [05:10<00:00,  6.04it/s]


In [ ]:
# Save models
torch.save(esp_theta_model.state_dict(), "contextual_model.pt")
pickle.dump(beta_t.cpu(), open("beta_t.pkl", "wb"))

In [ ]:
# SAMPLING
def sampling_imgs(list_numbers, return_steps=[], mode="ddpm"):
    x_T = torch.randn(len(list_numbers), 1, 28, 28).to(device)  # Just a random noise
    context = convert_text_label(list_numbers).to(
        device
    )  # The context i.e. Encoded string
    context_mask = torch.zeros_like(context).to(device)  # No mask

    x_t = x_T
    if len(return_steps) > 0:
        reverse_steps = []

    # Reverse diffusion
    for t in tqdm(range(T, 0, -1)):
        if t in return_steps:
            reverse_steps.append(x_t.detach().cpu().numpy())

        norm_t = torch.tensor([t / T]).repeat(len(list_numbers)).to(device)
        esp_theta = esp_theta_model(x_t, context, norm_t, context_mask)

        if mode == "ddpm":  # Probabilistic
            sigma = (
                torch.sqrt((1 - alphabar_t[t - 1]) / (1 - alphabar_t[t]))
                * sqrt_beta_t[t]
            )
            z = torch.randn(len(list_numbers), 1, 28, 28).to(device) if t > 1 else 0

            x_t = (
                oneover_sqrta[t] * (x_t - esp_theta * mab_over_sqrtmab[t]) + sigma * z
            )  # x_{t-1}

        elif mode == "ddim":  # Deterministic
            sigma = 1e-3
            z = torch.randn(len(list_numbers), 1, 28, 28).to(device) if t > 1 else 0
            x_t = (
                oneover_sqrta[t] * (x_t - esp_theta * sqrtmab[t])
                + esp_theta * torch.sqrt(1 - alphabar_t[t - 1] - sigma**2)
                + sigma * z
            )  # x_{t-1}

    if len(return_steps) > 0:
        reverse_steps.append(x_t.detach().cpu().numpy())  # Step 0
        return reverse_steps

    return x_t.detach().cpu().numpy()

In [ ]:
# Run testing DDIM
test_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
with torch.no_grad():
    sampled_imgs = sampling_imgs(test_numbers, mode="ddim")

# Visualize testing results
for idx in range(len(test_numbers)):
    print("Number '%s'" % n2t_mapping[test_numbers[idx]])

    max_v = np.max(sampled_imgs[idx][0])
    min_v = np.min(sampled_imgs[idx][0])
    img = (sampled_imgs[idx][0] - min_v) / (max_v - min_v)
    show_img(img)

100%|██████████| 300/300 [00:08<00:00, 35.96it/s]

Number 'zero'


Number 'one'


Number 'two'


Number 'three'


...

Number 'six'


Number 'seven'


Number 'eight'


Number 'nine'


Number 'ten'


In [ ]:
# Run testing DDPM
test_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
with torch.no_grad():
    sampled_imgs = sampling_imgs(test_numbers, mode="ddpm")

# Visualize testing results
for idx in range(len(test_numbers)):
    print("Number '%s'" % n2t_mapping[test_numbers[idx]])

    max_v = np.max(sampled_imgs[idx][0])
    min_v = np.min(sampled_imgs[idx][0])
    img = (sampled_imgs[idx][0] - min_v) / (max_v - min_v)
    show_img(img)

100%|██████████| 300/300 [00:08<00:00, 35.07it/s]

Number 'zero'


Number 'one'


Number 'two'


Number 'three'


...

Number 'six'


Number 'seven'


Number 'eight'


Number 'nine'


Number 'ten'


In [ ]:
test_numbers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
steps = list(range(T, -1, -50))

with torch.no_grad():
    reverse_steps = sampling_imgs(test_numbers, return_steps=steps)

print("\n=========================\n")
print("Number:", [n2t_mapping[x] for x in test_numbers])

for sidx, step in enumerate(steps):
    print("Step:", step)
    step_img = []

    for idx in range(len(test_numbers)):
        # print("Number '%s'" % n2t_mapping[test_numbers[idx]])

        max_v = np.max(reverse_steps[sidx][idx][0])
        min_v = np.min(reverse_steps[sidx][idx][0])
        img = (reverse_steps[sidx][idx][0] - min_v) / (max_v - min_v)
        step_img.append(img)

    step_img = np.concatenate(step_img, axis=-1)
    show_img(step_img)

100%|██████████| 300/300 [00:08<00:00, 34.34it/s]



Number: ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
Step: 300


Step: 250


Step: 200


Step: 150


Step: 100


Step: 50


Step: 0


In [ ]:
!cp contextual_model.pt "/content/drive/MyDrive/Colab Notebooks/Diffusion/contextual_model.pt"
!cp beta_t.pkl "/content/drive/MyDrive/Colab Notebooks/Diffusion/beta_t.pkl"